## E11 - 작사가 인공지능 만들기

In [1]:
# RNN: 문법을 처리하는 인공지능 중 하나, 순환신경망, 결과를 다시 입력으로 사용
# 시작과 끝을 알리는 신호가 필요하다.
# <start>: 문장 생성을 시작
# <end>: 문장 생성 완성

sentence = " 나는 밥을 먹었다 "

source_sentence = "<start>" + sentence
target_sentence = sentence + "<end>"

print("Source 문장:", source_sentence)
print("Target 문장:", target_sentence)

Source 문장: <start> 나는 밥을 먹었다 
Target 문장:  나는 밥을 먹었다 <end>


In [2]:
# wget https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
# 텐서플로우에서 제공하는 셰익스피어 연극 대본

import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os

# 파일을 읽기모드로 열어 봅니다.
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()   # 텍스트를 라인 단위로 끊어서 list 형태로 읽어옵니다.

print(raw_corpus[:9])    # 앞에서부터 10라인만 화면에 출력해 볼까요?

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [3]:
# 화자가 표기된 문장과 공백인 문장 제거

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


In [4]:
# 토큰화 중 문제점
# 문장부호가 붙은 경우, 대소문자 구분되는 경우, 특수문자가 붙은 경우
# 정규표현식을 이용하여 제거한다.
# 전치리 함수 구현부분

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 문장 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    # 패턴의 특수문자[?.!,¿]를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r'[" "]+', " ", sentence)
    # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'   # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다.
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [12]:
# raw_corpus를 전처리 하기
 
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

In [13]:
# tf.keras.preprocessing.text.Tokenizer 패키지를 이용하여 단어를 숫자 데이터로 변환시켜 준다.
# tensor란, https://rekt77.tistory.com/102

def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,    # 전체 단어의 개수 
        filters=' ',       # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding 메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fe67a59a090>


In [14]:
print(tensor[:3, :10])

# 데이터의 숫자는 tokenizer가 구축한 단어 사전의 인덱스

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


In [15]:
# 단어 사전 내용

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break
        
# 단어 사전의 내용을 확인해보면 2인덱스가 <start>이므로 위 tensor의 모든 행의 시작이 2인 걸 알 수 있다.

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [16]:
# 끝 단어를 없애면 소스 문장(X_train), 첫 단어를 없애면 타겟 문장(y_train)

src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]   # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


In [17]:
# tf.data.Dataset를 이용하여 데이터셋 객체를 생성한다.
# tensor형의 데이터셋을 만들어놨으므로 tf.data.Dataset.from_tensor_slices() 이용한다.

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

In [18]:
# Embedding -> RNN1(LSTM) -> RNN2(LSTM) -> Dense 로 구성된 모델 사용
# Embedding layer: 인덱스 값을 단어 사전에 해당 인덱스 번째의 워드 벡터로 바꿔 준다.

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256   # 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기 (예, 2일 경우 차갑다:[0.0, 1.0]
hidden_size = 1024     # LSTM layer의 hidden state의 차원수, 단어를 분석하는 일꾼수?
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [19]:
# model 빌드하기
# 약간 데이터를 돌려서 input shape를 알아내면 model.build()가 자동 호출된다.

for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

# shape=(256, 20, 7001) 확인
# 7001: Dense 레이어의 출력 차원수, 7001개의 각 단어의 확률, 높은 확률의 단어를 선택해야 하는 문제이기에
# dataset.take(1): 1개의 배치를 가져오는 것이므로 위에 지정한 256개의 배치 사이즈
# 20: return_sequences=True에 의한 것이며 LSTM이 입력 시퀀스 길이만큼 동일 길이의 시퀀스를 출력한다는 의미다.
# False면 1개의 벡터만 출력할 것이다.
# 길이는 데이터를 입력받으면서 알게 된 것이며 max_len이 20으로 지정되었다는 뜻이다.

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 6.19123093e-05,  1.44176622e-04,  5.34716455e-05, ...,
         -6.77949265e-06, -2.41072645e-04, -7.29690728e-05],
        [ 1.74911955e-04, -5.60510161e-05,  1.46098973e-04, ...,
         -9.45417705e-05, -3.83709063e-04, -1.14499628e-04],
        [ 6.09339186e-05, -3.26152192e-04,  1.94725901e-04, ...,
          1.99078335e-04, -6.62502192e-04, -3.48464062e-04],
        ...,
        [-2.09549791e-04, -2.87273806e-03, -1.23360194e-03, ...,
          6.22232154e-04, -1.62201002e-03, -5.66017523e-04],
        [-1.83532058e-04, -3.24721425e-03, -1.33686129e-03, ...,
          7.33096094e-04, -1.65901484e-03, -5.97535050e-04],
        [-1.50797918e-04, -3.57421930e-03, -1.39839842e-03, ...,
          8.61129491e-04, -1.68872159e-03, -5.97523584e-04]],

       [[ 6.19123093e-05,  1.44176622e-04,  5.34716455e-05, ...,
         -6.77949265e-06, -2.41072645e-04, -7.29690728e-05],
        [-6.57425626e-05,  5.69654840e-05,  9.

In [20]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [21]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 13s 141ms/step - loss: 3.4775
Epoch 2/30
93/93 [==============================] - 13s 141ms/step - loss: 2.8041
Epoch 3/30
93/93 [==============================] - 13s 142ms/step - loss: 2.7097
Epoch 4/30
93/93 [==============================] - 13s 142ms/step - loss: 2.6079
Epoch 5/30
93/93 [==============================] - 13s 142ms/step - loss: 2.5409
Epoch 6/30
93/93 [==============================] - 13s 143ms/step - loss: 2.4884
Epoch 7/30
93/93 [==============================] - 13s 143ms/step - loss: 2.4358
Epoch 8/30
93/93 [==============================] - 13s 143ms/step - loss: 2.3821
Epoch 9/30
93/93 [==============================] - 15s 161ms/step - loss: 2.3333
Epoch 10/30
93/93 [==============================] - 15s 166ms/step - loss: 2.2877
Epoch 11/30
93/93 [==============================] - 14s 153ms/step - loss: 2.2395
Epoch 12/30
93/93 [==============================] - 13s 144ms/step - loss: 2.1941
Epoch 13/30
9

In [22]:
# generate_text()는 모델에게 시작 문장을 전달하면 이 문장을 바탕으로 작문을 진행한다.

def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다.
        # 우리 모델이 예측한 마지막 단어([:, -1])가 바로 새롭게 생성한 단어가 됩니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [26]:
generate_text(model, tokenizer, init_sentence="<start> sky")

'<start> sky betwixt the firmament and it that i have <end> '